# * Adhoc-BB MKS by Province
    77 Province only

## Parameter

In [17]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
# import xlrd

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [32]:
# Input parameter

op_dir = 'output'
op_file = f'BB-MKS by Province_{str_curr_dt}'
v_param = dict(mth_start=202301, mth_end=202405)

print(f"\nParameter input...\n\n   -> op_dir: {op_dir}\n   -> op_file: {op_file}\n   -> mth_start: {v_param['mth_start']}\n   -> mth_end: {v_param['mth_end']}")


Parameter input...

   -> op_dir: output
   -> op_file: BB-MKS by Province_20240622
   -> mth_start: 202301
   -> mth_end: 202405


## ETL Process...

### Step 1 : Query from DB sources to Output file
    -> CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
    -> CDSAPPO.DIM_MOOC_AREA

In [29]:
# Connect : TDMDBPR
dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
conn = oracledb.connect(dsn)
print(f'\n{TDMDBPR_db} : Connected\n\nProcessing...')
cur = conn.cursor()


query = """
    /*** BB MKS by Province ***/

    WITH W_VAR (V_MONTH_START, V_MONTH_END) AS
    (
        SELECT :mth_start AS V_MONTH_START, :mth_end AS V_MONTH_END
        FROM DUAL
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_ORG AS 
    ( --7,436 row
        SELECT DISTINCT ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_R, TDS_RGM_CODE, ORGID_H, HOP_HINT, TDS_PROVINCE
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH, ORGID_HH, D_CLUSTER, CCAATT
        FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
        WHERE REMARK <> 'Dummy'
    )
    -----------------------------------------------------------------------------------------------------------------------

    , W_BB_MKS_RAWDATA AS 
    (
        SELECT TM_KEY_MTH, ISP
            , ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_R, TDS_RGM_CODE, ORGID_H, HOP_HINT, TDS_PROVINCE, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH, ORGID_HH, D_CLUSTER, EAST_FLAG
            , SUM(BASE_SUBS) BASE_SUBS, SUM(SUBS_CHG) SUBS_CHG, SUM(SUBS) SUBS
        FROM (
            SELECT A.TM_KEY_MTH, A.ISP, A.BASE_SUBS, A.SUBS_CHG, A.SUBS
                , B.ZONE_TYPE, B.ORGID_G, B.TDS_SGMD, B.ORGID_R, B.TDS_RGM_CODE, B.ORGID_H, B.HOP_HINT, B.TDS_PROVINCE, B.PROVINCE_CD, B.PROVINCE_ENG, B.PROVINCE_TH, B.ORGID_HH, B.D_CLUSTER
                , CASE WHEN (B.ORGID_G = 'GX3' AND B.HOP_HINT NOT LIKE 'SMP%') THEN 'Y' END EAST_FLAG
            FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT A
            INNER JOIN W_ORG B
                ON A.CCAATT = B.CCAATT
            WHERE A.TM_KEY_MTH BETWEEN (SELECT V_MONTH_START FROM W_VAR) AND (SELECT V_MONTH_END FROM W_VAR)
            AND ISP IN ('3BB','AIS','CAT','TOL','TOT')
        ) RAWDATA
        GROUP BY TM_KEY_MTH, ISP, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_R, TDS_RGM_CODE, ORGID_H, HOP_HINT, TDS_PROVINCE, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH, ORGID_HH, D_CLUSTER, EAST_FLAG
    ) -->> W_BB_MKS_RAWDATA
    -----------------------------------------------------------------------------------------------------------------------

    , W_BB_MKS_AGG_1 AS 
    (
        SELECT TM_KEY_MTH, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_R, TDS_RGM_CODE, ORGID_H, HOP_HINT, TDS_PROVINCE, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH, ORGID_HH, D_CLUSTER, EAST_FLAG
            , SUM(SUBS) AS SUBS_TOTAL
            , SUM(CASE WHEN ISP = 'TOL' THEN SUBS ELSE 0 END) AS SUBS_TOL
            , SUM(CASE WHEN ISP IN ('AIS', '3BB') THEN SUBS ELSE 0 END) AS SUBS_AIS_3BB
            , SUM(CASE WHEN ISP = '3BB' THEN SUBS ELSE 0 END) AS SUBS_3BB
            , SUM(CASE WHEN ISP = 'AIS' THEN SUBS ELSE 0 END) AS SUBS_AIS
            , SUM(CASE WHEN ISP IN ('CAT', 'TOT') THEN SUBS ELSE 0 END) AS SUBS_NT
            , SUM(CASE WHEN ISP = 'CAT' THEN SUBS ELSE 0 END) AS SUBS_CAT
            , SUM(CASE WHEN ISP = 'TOT' THEN SUBS ELSE 0 END) AS SUBS_TOT
        FROM W_BB_MKS_RAWDATA
        GROUP BY TM_KEY_MTH, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_R, TDS_RGM_CODE, ORGID_H, HOP_HINT, TDS_PROVINCE, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH, ORGID_HH, D_CLUSTER, EAST_FLAG
    ) -->> W_BB_MKS_AGG_1
    -----------------------------------------------------------------------------------------------------------------------

    -->> Results 77 Province
    SELECT SUBSTR(TM_KEY_MTH,1,4) AS TM_KEY_YR, TM_KEY_MTH, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
        , (SUM(SUBS_TOL) / SUM(SUBS_TOTAL)) * 100 AS MKS_TOL
        , (SUM(SUBS_AIS_3BB) / SUM(SUBS_TOTAL)) * 100 AS MKS_AIS_3BB
        , (SUM(SUBS_3BB) / SUM(SUBS_TOTAL)) * 100 AS MKS_3BB
        , (SUM(SUBS_AIS) / SUM(SUBS_TOTAL)) * 100 AS MKS_AIS
        , (SUM(SUBS_NT) / SUM(SUBS_TOTAL)) * 100 AS MKS_NT
        , SUM(SUBS_TOL) AS SUBS_TOL, SUM(SUBS_AIS_3BB) AS SUBS_AIS_3BB, SUM(SUBS_3BB) AS SUBS_3BB, SUM(SUBS_AIS) AS SUBS_AIS, SUM(SUBS_NT) AS SUBS_NT
        , SUM(SUBS_TOTAL) AS SUBS_TOTAL
    FROM W_BB_MKS_AGG_1
    GROUP BY TM_KEY_MTH, PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
    ORDER BY TM_KEY_MTH, PROVINCE_CD
"""


try:
    # Create Dataframe
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')
    cur.execute(query, (v_param['mth_start'], v_param['mth_end']))
    rows = cur.fetchall()
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
    print(f'\n   -> DataFrame : {df.shape[0]} rows, {df.shape[1]} columns')

    # # Generate CSV file
    # df.to_csv(f'{op_dir}/{op_file}.csv', index=False, encoding='TIS-620')
    # print(f'\n   -> Generate "{op_file}.csv" successfully')

    # Generate Excel file
    df.to_excel(f'{op_dir}/{op_file}.xlsx', sheet_name='Data', index=False)
    print(f'\n   -> Generate "{op_file}.xlsx" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')



TDMDBPR : Connected

Processing...

   -> Execute query... 2024-06-22, 14:25:04

   -> DataFrame : 1309 rows, 16 columns

   -> Generate "BB-MKS by Province_20240622.xlsx" successfully

TDMDBPR : Disconnected


In [30]:
''' Example DataFrame '''

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'METRIC_VALUE': 'mean', 'TM_KEY_MTH': 'nunique', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_MTH': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
# tmp_df

df#.tail(3)

tmp_df = df.groupby([''])

,TM_KEY_YR,TM_KEY_MTH,PROVINCE_CD,PROVINCE_ENG,PROVINCE_TH,MKS_TOL,MKS_AIS_3BB,MKS_3BB,MKS_AIS,MKS_NT,SUBS_TOL,SUBS_AIS_3BB,SUBS_3BB,SUBS_AIS,SUBS_NT,SUBS_TOTAL
0,2023,202301,10,Bangkok,กรุงเทพมหานคร,50.934741,33.854761,16.951158,16.903603,15.210498,880157,585013.3859,292917.5641,292095.8218,262838.8103,1.728009e+06
1,2023,202301,11,Samut Prakan,สมุทรปราการ,48.604797,38.911147,17.542964,21.368182,12.484056,175466,140471.3883,63331.0701,77140.3182,45068.1330,3.610055e+05
2,2023,202301,12,Nonthaburi,นนทบุรี,46.895078,41.713178,15.918781,25.794398,11.391744,175185,155827.0829,59467.4691,96359.6138,42555.9088,3.735680e+05
3,2023,202301,13,Pathum Thani,ปทุมธานี,47.346303,41.434864,16.451009,24.983856,11.218833,152339,133318.6636,52931.9093,80386.7543,36097.1331,3.217548e+05
4,2023,202301,14,Phra Nakhon Si Ayutthaya,พระนครศรีอยุธยา,29.445395,55.091824,33.283787,21.808036,15.462781,36389,68083.1880,41132.5347,26950.6533,19109.1046,1.235813e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,2024,202405,92,Trang,ตรัง,28.147596,47.739414,30.201218,17.538197,24.112990,10704,18154.3992,11484.9537,6669.4455,9169.7155,3.802811e+04
1305,2024,202405,93,Phatthalung,พัทลุง,29.075183,47.300788,29.985395,17.315393,23.624028,13564,22066.5126,13988.6272,8077.8854,11020.9563,4.665147e+04
1306,2024,202405,94,Pattani,ปัตตานี,22.645512,56.225115,29.004663,27.220451,21.129373,8258,20503.2674,10576.9525,9926.3149,7705.1188,3.646639e+04
1307,2024,202405,95,Yala,ยะลา,22.881968,55.588622,26.354158,29.234464,21.529410,7524,18278.5323,8665.7182,9612.8141,7079.2549,3.288179e+04
